In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob

os.getcwd()

'/home/jian/Projects/Smoothie_King/Analysis/TA_test_20190315'

In [2]:
latest_ta=pd.ExcelFile("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_20190311/SmoothieKing_TA_of_3_miles_zips_JL_2019-03-11.xlsx")
latest_ta.sheet_names

['summary_by_TA', 'output_TA_by_store', 'zip_TA', 'summary_by_store_count']

In [3]:
df_ta_by_store=latest_ta.parse("output_TA_by_store",dtype=str)
df_ta_by_store=df_ta_by_store[df_ta_by_store['status']=="Same"]
df_ta_by_store.shape

(784, 18)

In [4]:
weekly_sales_folder=glob.glob("/home/jian/Projects/Smoothie_King/Weekly_Sales/*.xlsx")
weekly_sales_folder

['/home/jian/Projects/Smoothie_King/Weekly_Sales/WeeklySalesSummaryData_20190128.xlsx',
 '/home/jian/Projects/Smoothie_King/Weekly_Sales/WeeklySalesSummaryData_20190204.xlsx',
 '/home/jian/Projects/Smoothie_King/Weekly_Sales/WeeklySalesSummaryData_20190211.xlsx',
 '/home/jian/Projects/Smoothie_King/Weekly_Sales/WeeklySalesSummaryData_20190218.xlsx',
 '/home/jian/Projects/Smoothie_King/Weekly_Sales/WeeklySalesSummaryData_20190225.xlsx',
 '/home/jian/Projects/Smoothie_King/Weekly_Sales/WeeklySalesSummaryData_20190304.xlsx',
 '/home/jian/Projects/Smoothie_King/Weekly_Sales/WeeklySalesSummaryData_20190311.xlsx',
 '/home/jian/Projects/Smoothie_King/Weekly_Sales/WeeklySalesSummaryData_20190121.xlsx',
 '/home/jian/Projects/Smoothie_King/Weekly_Sales/WeeklySalesSummaryData_20190114.xlsx']

In [5]:
df_all_weekly_sales_2019=pd.DataFrame()

test=pd.read_excel("/home/jian/Projects/Smoothie_King/Weekly_Sales/WeeklySalesSummaryData_20190114.xlsx")
cols=test.columns.tolist()


for file in weekly_sales_folder:
    df=pd.read_excel(file,dtype=str,usecols=cols)
    df['WeekEndDate']=df['WeekEndDate'].apply(lambda x:x[:10])
    df['NetSalesSmoothie']=df['NetSalesSmoothie'].astype(float)
    df['NetSalesRetail']=df['NetSalesRetail'].astype(float)
    df['WeekEndDate']=df['WeekEndDate'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    df_all_weekly_sales_2019=df_all_weekly_sales_2019.append(df)
    
df_all_weekly_sales_2019=df_all_weekly_sales_2019[(df_all_weekly_sales_2019['WeekEndDate']>=datetime.date(2019,1,6)) & (df_all_weekly_sales_2019['WeekEndDate']<=datetime.date(2019,3,9))]
week_end_dates_2019=sorted(df_all_weekly_sales_2019['WeekEndDate'].unique().tolist())

In [6]:
df_all_weekly_sales_2018=pd.read_excel("/home/jian/Projects/Smoothie_King/data/WeeklySalesSummaryData_111218.xlsx",dtype=str)
df_all_weekly_sales_2018['NetSalesSmoothie']=df_all_weekly_sales_2018['NetSalesSmoothie'].astype(float)
df_all_weekly_sales_2018['NetSalesRetail']=df_all_weekly_sales_2018['NetSalesRetail'].astype(float)

df_all_weekly_sales_2018['WeekEndDate']=df_all_weekly_sales_2018['WeekEndDate'].apply(lambda x: datetime.datetime.strptime(x[:10],"%Y-%m-%d").date())
week_end_dates_2018=[x - datetime.timedelta(days=52*7) for x in week_end_dates_2019]
df_all_weekly_sales_2018=df_all_weekly_sales_2018[df_all_weekly_sales_2018['WeekEndDate'].isin(week_end_dates_2018)]

In [7]:
sales_available_count_df=df_all_weekly_sales_2018.append(df_all_weekly_sales_2019)
sales_available_count_df=sales_available_count_df.groupby(['StoreNumber'])['WeekEndDate'].count().to_frame().reset_index()
print(sales_available_count_df['WeekEndDate'].max())

Weeks_Available_full=sales_available_count_df[sales_available_count_df['WeekEndDate']==18]
all_weeks_available_stores=Weeks_Available_full['StoreNumber'].unique().tolist()

18


In [8]:
all_sales_both_year=df_all_weekly_sales_2018.append(df_all_weekly_sales_2019)
all_sales_both_year=all_sales_both_year[all_sales_both_year['StoreNumber'].isin(all_weeks_available_stores)]

print(len(all_sales_both_year['StoreNumber'].unique()))

785


In [9]:
all_SameStore_sales_both_year=all_sales_both_year[all_sales_both_year['StoreNumber'].isin(df_ta_by_store['storenumber'])]
print(len(all_SameStore_sales_both_year['StoreNumber'].unique()))

765


In [10]:
# QC
aaa=all_sales_both_year[~all_sales_both_year['StoreNumber'].isin(df_ta_by_store['storenumber'])]
missed_stores=aaa['StoreNumber'].unique().tolist()
len(missed_stores)

20

In [11]:
store_list_latest=pd.read_excel("/home/jian/Projects/Smoothie_King/store_list_weekly/StoreList_20190311.xlsx",dtype=str)
store_list_latest[store_list_latest['storenumber'].isin(missed_stores)]
print(store_list_latest[store_list_latest['storenumber'].isin(missed_stores)].shape)
print(store_list_latest[store_list_latest['storenumber'].isin(missed_stores)]['status'].unique())

(20, 14)
['NonComp']


In [12]:
all_SameStore_sales_both_year['Total_Net_Sales']=all_SameStore_sales_both_year['NetSalesSmoothie']+all_SameStore_sales_both_year['NetSalesRetail']

all_SameStore_sales_both_year['Year']=all_SameStore_sales_both_year['WeekEndDate'].apply(lambda x: x.year)

sales_by_store_by_year=all_SameStore_sales_both_year.groupby(['StoreNumber','Year'])['Total_Net_Sales'].sum().to_frame().reset_index()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
sales_by_store_by_year.head(5)

,StoreNumber,Year,Total_Net_Sales
0,0000,2018,159149.68
1,0000,2019,97871.02
2,0002,2018,118417.05
3,0002,2019,122197.07
4,0003,2018,128778.99


In [14]:
sales_by_store_by_year_pivot=sales_by_store_by_year.pivot_table(index="StoreNumber",columns="Year").reset_index()
col_level_1_list=sales_by_store_by_year_pivot.columns.get_level_values(0).tolist()
col_level_2_list=sales_by_store_by_year_pivot.columns.get_level_values(1).tolist()
new_cols=[]
for i in range(len(col_level_1_list)):
    new_cols=new_cols+[str(col_level_1_list[i])+"_"+str(col_level_2_list[i])]
sales_by_store_by_year_pivot.columns=new_cols
sales_by_store_by_year_pivot=sales_by_store_by_year_pivot.rename(columns={"StoreNumber_":"storenumber"})
# sales_by_store_by_year_pivot.columns=sales_by_store_by_year_pivot.columns.get_level_values([0,1])
# sales_by_store_by_year_pivot.columns

In [15]:
df_ta_by_store=df_ta_by_store[['storenumber','TA']].drop_duplicates()

sales_by_store_by_year_pivot=pd.merge(sales_by_store_by_year_pivot,df_ta_by_store,on="storenumber",how="left")

In [16]:
sales_by_store_by_year_pivot.head(2)

,storenumber,Total_Net_Sales_2018,Total_Net_Sales_2019,TA
0,0000,159149.68,97871.02,"17_['0000', '0002', '0003', '0006', '0010', '0..."
1,0002,118417.05,122197.07,"17_['0000', '0002', '0003', '0006', '0010', '0..."


In [17]:
output_df_store_count=sales_by_store_by_year_pivot.groupby('TA')['storenumber'].count().to_frame().reset_index().rename(columns={"storenumber":"same_store_count"})
output_df_store_list=sales_by_store_by_year_pivot.groupby('TA')['storenumber'].apply(list).to_frame().reset_index().rename(columns={"storenumber":"same_store_list"})
output_df_store_sales=sales_by_store_by_year_pivot.groupby('TA')['Total_Net_Sales_2018','Total_Net_Sales_2019'].sum().reset_index()


output_df=pd.merge(output_df_store_count,output_df_store_list,on="TA",how="outer")
output_df=pd.merge(output_df,output_df_store_sales,on="TA",how="outer")


In [18]:
df_TA_info=latest_ta.parse("summary_by_TA",dtype=str)
df_TA_info=df_TA_info[['TA','Primary_DMA','DMA_list']]
df_TA_info['DMA_list']=df_TA_info['DMA_list'].apply(lambda x: eval(x.replace("nan, ","")))

In [19]:
output_df=pd.merge(output_df,df_TA_info,on="TA",how="left")
output_df=output_df[['TA','Primary_DMA','same_store_count','Total_Net_Sales_2019','Total_Net_Sales_2018','DMA_list']]

output_df['DMA_count']=output_df['DMA_list'].apply(lambda x: len(x))

In [31]:
writer=pd.ExcelWriter("/home/jian/Projects/Smoothie_King/Analysis/TA_test_20190315/SK_Same_store_sales_comp_by_TA_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_df.to_excel(writer,"output",index=False)
sales_by_store_by_year_pivot.to_excel(writer,"Same_store_inclusion",index=False)
all_SameStore_sales_both_year.to_excel(writer,"Same_store_by_week",index=False)
writer.save()

In [29]:
print("2018_Min: ",all_SameStore_sales_both_year[all_SameStore_sales_both_year['Year']==2018]['WeekEndDate'].min())
print("2018_Max: ",all_SameStore_sales_both_year[all_SameStore_sales_both_year['Year']==2018]['WeekEndDate'].max())

print("2019_Min: ",all_SameStore_sales_both_year[all_SameStore_sales_both_year['Year']==2019]['WeekEndDate'].min())
print("2019_Max: ",all_SameStore_sales_both_year[all_SameStore_sales_both_year['Year']==2019]['WeekEndDate'].max())


2018_Min:  2018-01-13
2018_Max:  2018-03-10
2019_Min:  2019-01-12
2019_Max:  2019-03-09
